<a href="https://colab.research.google.com/github/KonradGonrad/PyTorch-deep-learning/blob/main/03_PyTorch_computer_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch Computer Vision

## 0. Computer vision libaries in PyTorch

* [`torchvision`](https://pytorch.org/vision/stable/index.html) - base domain libary for Pytorch computer vision
* `torchvision.datasets` - databases and data loading functions for computer vision
* `torchvision.models` - get pretrained computer vision models that can leverage your own problems
* `torchvision.transforms` - functions for manipulating your vision data (images) to be suitable for use with and ML model
* `torch.utils.data.Dataset` - Base dataset class for PyTorch
* `torch.utils.data.Dataloader` - Creates a Python iterable over a dataset


In [ ]:
# Impoprt PyTorch
import torch
from torch import nn

# Import torchvision
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

# Import matplotlib
import matplotlib.pyplot as plt

# Import pandas (maybe will be needed)
import pandas as pd

# Check version
print(torch.__version__)
print(torchvision.__version__)

## 1. Getting a dataset

FashionMNIST is a dataset of greyscale images of clothingf

In [ ]:
train_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
    target_transform=None
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform=None
)

In [ ]:
print(f"length of training data: {len(train_data)}")
print(f"length of testing data: {len(test_data)}")

In [ ]:
data, label = train_data[0]
print(data.shape, label)

In [ ]:
target_classes = train_data.classes
target_classes

In [ ]:
classes_to_idx = train_data.class_to_idx
classes_to_idx

In [ ]:
train_data.targets

### 1.1 Check Input and output shapes of data

In [ ]:
print(f"data of shape {data.shape} -> [Color, Height, Weight] equals {target_classes[label]}")

### 1.2 Visualizing our data


In [ ]:
torch.manual_seed(42) # Setting up a random seed to have same outputs
fig = plt.figure(figsize=(9, 9)) # Creating a figure, where we'll put our subplots
rows, cols = [4, 4] # Setting up dimension, there is 4x4 what gives us 16 pics
for i in range(1, rows*cols + 1): # Setting up loop to put our imgs into figure
  random_idx = torch.randint(0, len(train_data), size=[1]).item() # Getting random index from 0 to train_data max index
  img, label = train_data[random_idx] # assign img and label to variables
  fig.add_subplot(rows, cols, i) # Adding subplot to figure at i index
  plt.imshow(img.squeeze(), cmap='gray') # Creating image plot which is added to figure
  plt.title(target_classes[label]) # Adding title
  plt.axis(False) # Removing aaxis, beacouse it's useless in our case

Do you think these items of clothing (images) could be modelled with pure linear lines? Or do you think we'll need non-linearities

## 2. Prepare dataloader

Right now, our data is in the form of PyTorch datasets

DataLoader turns our dataset into Python iterable

More specifically, we want to turn our data into batches (or mini-batches).

Why would we do this?

1. It is more computationally efficient, as in, your computing hardware may not be able to look (store in memory) at 60000 images in one hit. So we break it down to 32 images at a time (batch size of 32)
2. It gives our neural network more chances to update its gradients per epoch

for more on [mini-batches](https://www.bilibili.com/video/BV1RE411Z7YW/)

In [ ]:
train_data, test_data

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)

train_dataloader, test_dataloader

In [ ]:
print(f"train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}...")
print(f"test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}...")

In [ ]:
train_features_batch, train_labels_batch = next(iter(train_dataloader))
print(f"train_feature_batch shape: {train_features_batch.shape} and train_labels_batch shape: {train_labels_batch.shape}")

In [ ]:
random_idx = torch.randint(0, len(train_features_batch), size=[1]).item()
img, title = train_features_batch[random_idx], train_labels_batch[random_idx]
plt.imshow(img.squeeze(), cmap='gray')
plt.title(target_classes[label])
plt.axis(False)
print(f"image shape: {img.shape}")
print(f"Label: {label}, label size: {title.shape}")

## 3.0 Model 0: Build a baseline model

In [ ]:
# Create a flatten layer
flatten_layer = nn.Flatten()

# Get a single sample
x = train_features_batch[0]
x

# Flatten the sample
output = flatten_layer(x)
output

In [ ]:
from torch import nn
class FashionMNISTModelV0(nn.Module):
  def __init__(self,
               input_features: int,
               hidden_layers: int,
               output_features: int):
    super().__init__()
    self.layer_1 = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_features, out_features=hidden_layers),
        nn.Linear(in_features=hidden_layers, out_features=output_features)
    )

  def forward(self, x):
    return self.layer_1(x)




In [ ]:
model_0 = FashionMNISTModelV0(input_features=28*28,
                              hidden_layers=10,
                              output_features=len(target_classes))

In [ ]:
model_0(x).shape

In [ ]:
x_dummy = torch.rand([1, 1, 28, 28])
model_0(x_dummy).shape